In [ ]:
import os, sys
import numpy as np

from pathlib import Path

from astropy.io import fits, ascii
from astropy.table import Table, Column, vstack, hstack, join
from astropy.cosmology import FlatLambdaCDM

from scipy import interpolate
import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D

%matplotlib inline

sys.path.append("./py")
#from utils import *

sys.path.append("/Users/aberti/Desktop/research")
from plotutils import get_corners, fig_labels, get_colors, plot_settings
plt.rcParams.update(**plot_settings)

plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["font.family"] = "STIXGeneral"

from params import BASEDIR, DATADIR, SIMDIR, H0, Om0
h = H0/100

cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)

import Corrfunc
from Corrfunc.theory.wp import wp as wp_corrfunc

#-- bins for clustering (data and mocks)
from params import nbins, rp_min, rp_max, rp_bins, rp_mids, bin_file_comoving, bin_file
from params import bin_file_test as bin_file_zhou


In [ ]:
b = ascii.read(bin_file)
rp_bins = np.concatenate([b["col1"],[b["col2"][-1]]])
rp_mids = 0.5*(rp_bins[1:] + rp_bins[:-1])

b = ascii.read(bin_file_comoving)
rp_bins_com = np.concatenate([b["col1"],[b["col2"][-1]]])
rp_mids_com = 0.5*(rp_bins_com[1:] + rp_bins_com[:-1])

b = ascii.read(bin_file_zhou)
rp_bins_zhou = np.concatenate([b["col1"],[b["col2"][-1]]])
rp_mids_zhou = 0.5*(rp_bins_zhou[1:] + rp_bins_zhou[:-1])


In [ ]:
# print("default\tcom\tzhou")

# for i in range(np.min([len(rp_bins),len(rp_bins_com),len(rp_bins_zhou)])):
#     print(f"{rp_bins[i]:.4f}\t{rp_bins_com[i]:.4f}\t{rp_bins_zhou[i]:.4f}")

# print("\ndefault\tcom\tzhou")

# for i in range(np.min([len(rp_mids),len(rp_mids_com),len(rp_mids_zhou)])):
#     print(f"{rp_mids[i]:.4f}\t{rp_mids_com[i]:.4f}\t{rp_mids_zhou[i]:.4f}")  
    

# Rockstar ascii file parser stuff

In [ ]:
# header = "scale(0) id(1) desc_scale(2) desc_id(3) num_prog(4) pid(5) upid(6) desc_pid(7) phantom(8) sam_mvir(9) mvir(10) rvir(11) rs(12) vrms(13) mmp?(14) scale_of_last_MM(15) vmax(16) x(17) y(18) z(19) vx(20) vy(21) vz(22) Jx(23) Jy(24) Jz(25) Spin(26) Breadth_first_ID(27) Depth_first_ID(28) Tree_root_ID(29) Orig_halo_ID(30) Snap_num(31) Next_coprogenitor_depthfirst_ID(32) Last_progenitor_depthfirst_ID(33) Last_mainleaf_depthfirst_ID(34) Tidal_Force(35) Tidal_ID(36) Rs_Klypin(37) Mmvir_all(38) M200b(39) M200c(40) M500c(41) M2500c(42) Xoff(43) Voff(44) Spin_Bullock(45) b_to_a(46) c_to_a(47) A[x](48) A[y](49) A[z](50) b_to_a(500c)(51) c_to_a(500c)(52) A[x](500c)(53) A[y](500c)(54) A[z](500c)(55) T/|U|(56) M_pe_Behroozi(57) M_pe_Diemer(58) Macc(59) Mpeak(60) Vacc(61) Vpeak(62) Halfmass_Scale(63) Acc_Rate_Inst(64) Acc_Rate_100Myr(65) Acc_Rate_1*Tdyn(66) Acc_Rate_2*Tdyn(67) Acc_Rate_Mpeak(68) Mpeak_Scale(69) Acc_Scale(70) First_Acc_Scale(71) First_Acc_Mvir(72) First_Acc_Vmax(73) Vmax\@Mpeak(74) Tidal_Force_Tdyn(75) Log_(Vmax/Vmax_max(Tdyn;Tmpeak))(76) Time_to_future_merger(77) Future_merger_MMP_ID(78)"

# for i in range(len(header.split(" "))):
#     print(f"{i}\t{header.split(' ')[i].split('(')[0]}")
          

In [ ]:
# def parse_rockstar_ascii(halocat_filepath, quiet=True, N_chunks="all", chunk_length=50000, save=False,
#                          cols=[1,5,6,10,11,12,15,16,17,18,19,20,21,22,59,60,61,62,63,69,70,71,72,73,74]):
#     """
#     Parses raw ascii Rockstar halo files and saves selected columns in npy/table format
#     """
#     if N_chunks != "all":
#         assert( (type(N_chunks)==int) & (N_chunks > 0) )
#         if not quiet:
#             print(f"Processing {N_chunks} X {chunk_length} lines of {halocat_filepath}...")
#     else:
#         assert(os.path.exists(halocat_filepath))
#         if not quiet:
#             print(f"Reading all of {halocat_filepath}...")
#     with open(halocat_filepath, "r") as f:
#         #-- parse header
#         line0 = f.readline()
#         line0 = line0.strip()
#         colnames = line0.split()

#         #-- metadata for table
#         names = [colnames[i][:-4].lower() if (colnames[i][-4]=="(") else colnames[i][:-3].lower() for i in cols]
#         dtype = [int if (("id" in colnames[i].lower()) or ("?" in colnames[i])) else float for i in cols]

#         if not quiet:
#             for i in range(len(names)):
#                 print(f"{cols[i]}\t{names[i]}")

#         #-- skip over column definitions
#         for i in range(63):
#             line = f.readline()

#         chunks = []
#         chunk_num = 0
#         while line != "":
#             this_chunk = []
#             for n in range(chunk_length):
#                 line = f.readline().strip()
#                 if line=="":
#                     break
#                 row = np.asarray(line.split())
#                 this_chunk.append([float(i) if (("e" in i) or ("." in i)) else int(i) for i in [row[j] for j in cols]])     
#             this_chunk = Table( np.asarray(this_chunk), names=names, dtype=dtype )
#             mask = this_chunk["mvir"] >= 1e11
#             # print(len(this_chunk), len(this_chunk[mask]))
#             chunks.append( this_chunk[mask] )
#             if not quiet and chunk_num%100==0:
#                 print(chunk_num)
#             chunk_num += 1
#             if (N_chunks != "all") & (chunk_num==N_chunks):
#                 break
#         f.close()
#         halocat = vstack(chunks)
#         if not quiet:
#             print("Your simulation snapshot is ready!")

#         save_dir_npy = "/" + os.path.join(*halocat_filepath.split("/")[1:-1])
#         fname_npy    = "{}p{}_mvir1e11.npy".format(*halocat_filepath.split("/")[-1].split(".")[:-1])
#         if not quiet:
#             print(f"Saving {save_dir_npy}/{fname_npy}...")
#         np.save(f"{save_dir_npy}/{fname_npy}", halocat)

#         return halocat

    

In [ ]:
# %%time

# input_fname = "/Users/aberti/Desktop/research/sims/mdpl2/CATALOGS/hlist_0.61420.list"

# cols = [1,5,6,10,11,12,17,18,19,62,70]

# halocat = parse_rockstar_ascii(input_fname, quiet=False, N_chunks="all", cols=cols, save=True)


# Load halo catalog

In [ ]:
%%time

# from numpy.lib import recfunctions as rf

# redshift = 0.42531
# halocat_fname = "/Users/aberti/Desktop/research/sims/mdpl2/CATALOGS/hlist_0p70160_mvir1e11.npy"

redshift = 0.52323
halocat_fname = "/Users/aberti/Desktop/research/sims/mdpl2/CATALOGS/hlist_0p65650_mvir1e11.npy"

# redshift = 0.62813
# halocat_fname = "/Users/aberti/Desktop/research/sims/mdpl2/CATALOGS/hlist_0p61420_mvir1e11.npy"

halocat = Table(np.load(halocat_fname))


# Halo catalog processing
## Trim columns and Mvir limit (if needed)

In [ ]:
%%time

# halocat = halocat[halocat["mvir"] >= 1e11]
# halocat = Table(halocat)

for i,c in enumerate(halocat.colnames):
    print(f"{i}\t{c}")


In [ ]:
# for col in np.array(halocat.colnames)[[6,7,11,12,13,15,16,18,19,20,21]]:
#     print(col)
#     halocat.remove_column(col) 
    

## Add parent Mvir to halo catalog (if needed)

In [ ]:
%%time

from utils import crossmatch

# halocat.remove_column("parent_mvir")
if "parent_mvir" not in halocat.colnames:
    print("Adding parent_mvir column...")
    
    #-- add parent_mvir to halocat
    #-- default value is mvir but will overwrite that below for subhalos
    halocat.add_column(Column(data=halocat["mvir"].data, name="parent_mvir"))
    halo_id   = halocat["id"]   # unique
    halo_upid = halocat["upid"] # not unique

    upid_idx, id_idx = crossmatch(halo_upid, halo_id, skip_bounds_checking=True)

    halocat["parent_mvir"][upid_idx] = halocat["mvir"][id_idx]

    parent_halo_mask = (halocat["upid"] == -1)
    halocat["parent_mvir"][parent_halo_mask] = halocat["mvir"][parent_halo_mask]

    np.save(halocat_fname, halocat)


In [ ]:
# len(halocat[halocat["mvir"] == halocat["parent_mvir"]])/len(halocat)

# halocat


In [ ]:
# bins = np.arange(11,16,0.05)

# plt.hist(np.log10(halocat["mvir"]), bins=bins, alpha=0.5)
# plt.hist(np.log10(halocat["parent_mvir"])[halocat["upid"]==-1], bins=bins, alpha=0.5)
# plt.hist(np.log10(halocat["parent_mvir"])[halocat["upid"]!=-1], bins=bins, alpha=0.5)

# plt.semilogy()
# plt.ylim(10,1e7)
# plt.show()


In [ ]:
# rf.get_names(halocat.dtype)


In [ ]:
# %%time

# halocat = rf.rename_fields(halocat, {"vmax":"parent_mvir"})

# rf.get_names(halocat.dtype)


In [ ]:
# %%time

# fields = ("rs","rvir","scale_of_last_mm","macc","mpeak","vacc","vpeak","halfmass_scale",
#           "mpeak_scale","acc_scale","first_acc_scale","first_acc_mvir","first_acc_vmax",
#           "vmax\\@mpeak")#,"vmax")

# rf.drop_fields(halocat, fields[0])


# Populate halo catalog with mock LRGs according to Zhou et al. 2021 HOD

In [ ]:
from scipy import special as sp

def ncen_mvir(log_mvir, log_mmin, sigma_logm):
    pcen = (0.5*( 1. + sp.erf( (log_mvir - log_mmin) / sigma_logm ) ))
    idx = np.where(np.array([str(i) for i in pcen])=="nan")[0]
    pcen[idx] = 0
    return pcen

def nsat_mvir(log_mvir, log_m0, log_m1, alpha):
    mean_nsat = (( (10**log_mvir - 10**log_m0) / 10**log_m1 )**alpha)
    idx = np.where(np.array([str(i) for i in mean_nsat])=="nan")[0]
    mean_nsat[idx] = 0
    return mean_nsat


zhou_data = ascii.read("zhou_lrg_hod.csv")

these_zhou_params = zhou_data[np.round(zhou_data["zsim"],1)==np.round(redshift,1)]

log_mmin   = these_zhou_params["log_Mmin_6d"]
sigma_logm = these_zhou_params["sigma_logM_6d"]

log_m0     = these_zhou_params["logM0_6d"]
log_m1     = these_zhou_params["logM1_6d"]
alpha      = these_zhou_params["alpha_6d"]


In [ ]:
halocat = halocat[halocat["mvir"] >= 10**(12)]

# log_mvir_bins = np.arange(12,np.round(np.log10(np.max(halocat["mvir"])),2)+0.1,0.1)

log_mvir_bins = np.arange(12,15.01,0.1)
log_mvir_bins = np.concatenate( [log_mvir_bins,[np.log10(np.max(halocat["mvir"]))]] )

log_mvir_cens = np.round(0.5*(log_mvir_bins[1:] + log_mvir_bins[:-1]),3)

# log_mvir_bins

# quantiles = np.arange(0,1.01,0.01)
# log_mvir_bins = np.quantile(np.log10(halocat["mvir"]), quantiles)

fig, ax = plt.subplots(1, 1, figsize=(8,6))

ax.hist(np.log10(halocat["mvir"]), bins=log_mvir_bins, alpha=0.3, color="orangered", label="Mvir (all halos)")
# ax.hist(np.log10(halocat["parent_mvir"])[halocat["upid"]==-1], bins=log_mvir_bins, alpha=0.3, color="blue", label="parent Mvir (host halos)")
ax.hist(np.log10(halocat["parent_mvir"])[halocat["upid"]!=-1], bins=log_mvir_bins, alpha=0.5, color="gray", label="parent Mvir (subhalos)")

ax.legend()
ax.set_ylim(1,1e6)
ax.set_xlabel("log ( parent halo Mvir )")

ax.semilogy()
plt.show()



In [ ]:
%%time

is_host_halo = halocat["upid"] == -1
is_subhalo   = ~is_host_halo

total_host_halos_by_parent_mvir,_ = np.histogram(np.log10(halocat["parent_mvir"][is_host_halo]), bins=log_mvir_bins)
total_subhalos_by_parent_mvir,_   = np.histogram(np.log10(halocat["parent_mvir"][is_subhalo]), bins=log_mvir_bins)

prob_has_cen_by_parent_mvir = ncen_mvir(log_mvir_cens, log_mmin, sigma_logm)
mean_num_sat_by_parent_mvir = nsat_mvir(log_mvir_cens, log_m0, log_m1, alpha)


In [ ]:
%%time

total_occ_host_halos_by_parent_mvir = np.round(prob_has_cen_by_parent_mvir*total_host_halos_by_parent_mvir,0)
total_occ_subhalos_by_parent_mvir   = np.round(mean_num_sat_by_parent_mvir*total_host_halos_by_parent_mvir,0)


In [ ]:
# total_host_halos_by_parent_mvir
# total_subhalos_by_parent_mvir

# prob_has_cen_by_parent_mvir
# mean_num_sat_by_parent_mvir

# print(np.sum(total_occ_host_halos_by_parent_mvir) / np.sum(total_host_halos_by_parent_mvir))
# print(np.sum(total_occ_subhalos_by_parent_mvir) / np.sum(total_subhalos_by_parent_mvir))


In [ ]:
%%time

col = "galaxy"
if col in halocat.colnames:
    halocat.remove_column(col)
halocat.add_column(Column( np.zeros(len(halocat),dtype=bool), name="galaxy"))

log_parent_mvir = np.log10(halocat["parent_mvir"])
         
for i in range(len(log_mvir_bins)-1):
    
    log_parent_mvir_mask = (log_parent_mvir >= log_mvir_bins[i]) & (log_parent_mvir < log_mvir_bins[i+1])

    host_halos_this_parent_mvir_bin = halocat[log_parent_mvir_mask & is_host_halo]
    subhalos_this_parent_mvir_bin   = halocat[log_parent_mvir_mask & is_subhalo]

    n_host_halos_this_parent_mvir_bin = len(host_halos_this_parent_mvir_bin)
    n_subhalos_this_parent_mvir_bin   = len(subhalos_this_parent_mvir_bin)

    target_occ_host_halos = int(total_occ_host_halos_by_parent_mvir[i])
    target_occ_subhalos   = int(total_occ_subhalos_by_parent_mvir[i])
    
    if (target_occ_host_halos > 0) | (target_occ_subhalos > 0) | (i%100==0):
        print(f"{log_mvir_cens[i]}\t{target_occ_host_halos}\t{target_occ_subhalos}")
    
    indices = np.where(log_parent_mvir_mask)[0]            
    
    #-- populate host halos with central galaxies
    if target_occ_host_halos > 0:
        print(n_host_halos_this_parent_mvir_bin, target_occ_host_halos)
        #-- check that target occupation number does not exceed number of available host halos
        # assert(n_host_halos_this_parent_mvir_bin >= target_occ_host_halos)
        indices = np.where(log_parent_mvir_mask & is_host_halo)[0]            
        if (target_occ_host_halos < n_host_halos_this_parent_mvir_bin):
            occ_host_halos = indices[np.random.choice(len(indices), target_occ_host_halos, replace=False)]
            halocat["galaxy"][occ_host_halos] = True
        else:
            halocat["galaxy"][indices] = True

    #-- populate host halos with satellite galaxies
    if target_occ_subhalos > 0:
        print(n_subhalos_this_parent_mvir_bin, target_occ_subhalos)
        #-- check that target occupation number does not exceed number of available subhalos
        # assert(n_subhalos_this_parent_mvir_bin >= target_occ_subhalos)
        indices = np.where(log_parent_mvir_mask & is_subhalo)[0]            
        if (target_occ_subhalos < n_subhalos_this_parent_mvir_bin):
            occ_subhalos = indices[np.random.choice(len(indices), target_occ_subhalos, replace=False)]
            halocat["galaxy"][occ_subhalos] = True
        else:
            halocat["galaxy"][indices] = True

            

In [ ]:
galcat = halocat[halocat["galaxy"]==True]

is_cen = galcat["upid"] == -1
is_sat = ~is_cen

print(len(galcat[is_sat]) / len(galcat))


In [ ]:
print(len(galcat)/(1000.**3))


## Plot target vs achieved halo occupation numbers

In [ ]:
c_cen = "blue"
c_sat = "darkorange"

fig, axes = plt.subplots(2, 1, figsize=(10,10), sharex=True)

ax = axes[0]
ax.plot(log_mvir_cens, total_host_halos_by_parent_mvir, color=c_cen, ls="--", label="all host halos")
ax.plot(log_mvir_cens, total_occ_host_halos_by_parent_mvir, color=c_cen, label="occupied host halos")

ax.plot(log_mvir_cens, total_subhalos_by_parent_mvir, color=c_sat, ls="--", label="all subhalos")
ax.plot(log_mvir_cens, total_occ_subhalos_by_parent_mvir, color=c_sat, label="occupied subhalos")

ax.plot(log_mvir_cens, np.histogram(np.log10(galcat["parent_mvir"][is_cen]), bins=log_mvir_bins)[0], color=c_cen, lw=12, alpha=0.2)
ax.plot(log_mvir_cens, np.histogram(np.log10(galcat["parent_mvir"][is_sat]), bins=log_mvir_bins)[0], color=c_sat, lw=12, alpha=0.3)

ax.legend(markerfirst=False)
ax.set_ylabel("number of halos")

ax.semilogy()
ax.set_xlim(12,15.2)
#ax.set_ylim(1, 1e7)

ax = axes[1]
ax.plot(log_mvir_cens, total_occ_host_halos_by_parent_mvir / total_host_halos_by_parent_mvir, color=c_cen,
       label="host halos")
ax.plot(log_mvir_cens, total_occ_subhalos_by_parent_mvir / total_subhalos_by_parent_mvir, color=c_sat,
       label="subhalos")

ax.set_ylim(1e-3,2)

ax.set_xlabel(r"$\log\ (\ $host halo $M_{\rm vir}\ )$")
ax.set_ylabel("occupation fraction")
ax.legend()
ax.semilogy()

plt.tight_layout()
plt.subplots_adjust(hspace=0.025)
plt.show()


## Clustering

In [ ]:
from scipy.signal import savgol_filter
boxsize  = 1000.
nthreads = 2
pimax    = 150.

extra_mask = np.ones(len(galcat), dtype=bool)
# extra_mask = galcat["mvir"] >= 10**(12)

fig, ax = plt.subplots(1, 1, figsize=(9,8))

ax.set_xlim(0.03,40)
ax.set_ylim(50,300)
ax.tick_params(axis="both", which="major", labelsize=24)

ax.set_xlabel(fig_labels["rp"], fontsize=26)
ax.set_ylabel(r"$r_{\rm p}\times\omega_{\rm p}(r_{\rm p})\ \left(h^{-1}{\rm Mpc}^2\right)$", fontsize=26)
    
b = ascii.read(bin_file_zhou)
rp_bins = np.concatenate([b["col1"],[b["col2"][-1]]])
rp_mids = 0.5*(rp_bins[1:] + rp_bins[:-1])

out = []
for (u,v,w) in (("x","y","z"),("y","z","x"),("z","x","y")):
    xx = galcat[u][extra_mask]
    yy = galcat[v][extra_mask]
    zz = galcat[w][extra_mask]

    wp_mod = wp_corrfunc(boxsize, pimax, nthreads, bin_file_zhou, xx, yy, zz, output_rpavg=False)["wp"]
    ax.plot(rp_mids, rp_mids*wp_mod, color="blue", ls="--", alpha=0.5)#, label=r"model ($r_{\rm p} > 0.10\ h^{-1}{\rm Mpc}$)", lw=2)

#wp_mean = np.mean(np.array(out).T, axis=1)

# ax.plot(rp_mids, rp_mids*wp_mean, color="blue")#, label=r"model ($r_{\rm p} > 0.10\ h^{-1}{\rm Mpc}$)", lw=2)
# ax.scatter(rp_mids, rp_mids*wp_mean, color="blue")#, label=r"model ($r_{\rm p} > 0.10\ h^{-1}{\rm Mpc}$)", lw=2)
# ax.legend()#numpoints=2, handlelength=2, fontsize=24, loc=2, handletextpad=0.2, labelspacing=0.25)

ax.semilogx()
ax.set_yticks(np.arange(50, 301, 25))
ax.grid()
ax.set_title(r"$z_{\rm sim} = $" + f"{redshift:.3f}", fontsize=24)

ax.fill_between((ax.get_xlim()[0],0.1),ax.get_ylim()[1]*np.ones(2), color="gray", alpha=0.2)
ax.fill_between((20,ax.get_xlim()[1]),ax.get_ylim()[1]*np.ones(2), color="gray", alpha=0.2)

# figname = f"{BASEDIR}/figures/zhou_zsim{str(np.round(redshift,3)).replace('.','p')}.png"

# print(f"Saving {figname}...")
# plt.savefig(figname, dpi=200, bbox_inches="tight", pad_inches=0.1)

# plt.show()
    


# Comparison of radial LRG satellite distributions (Zhou+ 2021 vs my model)

In [ ]:
# #-- load my model galaxy catalog with LRG flags
# galcat = Table(np.load(f"{BASEDIR}/mocks/mdpl2/vpeak/south/color_scatter/zsnap0p42531_zmaglim20p7_Mzlimn21p6_rpmin0p1Mpch_brightest-mag-bin-rp1Mpch_LRG-flagged.npy"))

# #-- isolate just LRGs
# lrgcat = galcat[galcat["LRG_opt"]==True]



In [ ]:
# %%time

# #-- get satellite data for all central LRGs with satellites (Zhou model or mine)

# host_halo_ids = lrgcat["halo_id"][(lrgcat["upid"]==-1)]
# sats = []

# for i,hhid in enumerate(host_halo_ids):
#     these_sats = lrgcat[lrgcat["upid"]==hhid]
#     if len(these_sats) > 0:
#         sats.append(these_sats)
#     if (i%10000==0) & (i > 1):
#         print(f"{i} / {len(host_halo_ids)}")
        

In [ ]:
#-- load my saved LRG satellite data
sats = np.load(f"{BASEDIR}/sats_Mz_lrg-opt_z0p42531.npy")

#-- load Zhou! 2021 saved LRG satellite data
sats_zhou = np.load(f"{BASEDIR}/sats_zhou_lrg_z0p42531.npy")


In [ ]:
#-- galcat definition depends on whether working with Zhou+2021 model or my model

# galcat = halocat[halocat["galaxy"]==True]


In [ ]:
%%time

#-- compile radial LRG satellite distances for Zhou+ 2021
radii_zhou = []

for these_sats in sats_zhou[::5]:
    this_cen = galcat[galcat["halo_id"]==these_sats["upid"][0]]
    x0,y0,z0 = this_cen["x"],this_cen["y"],this_cen["z"]
    xx,yy,zz = these_sats["x"],these_sats["y"],these_sats["z"]
    rr = np.sqrt((xx - x0)**2 + (yy - y0)**2 + (zz - z0)**2)

    radii_zhou.append(rr)

radii_zhou = np.concatenate(radii_zhou)


In [ ]:
%%time

#-- compile radial LRG satellite distances for my model
radii = []

for these_sats in sats:
    this_cen = lrgcat[lrgcat["halo_id"]==these_sats["upid"][0]]
    x0,y0,z0 = this_cen["x"],this_cen["y"],this_cen["z"]
    xx,yy,zz = these_sats["x"],these_sats["y"],these_sats["z"]
    rr = np.sqrt((xx - x0)**2 + (yy - y0)**2 + (zz - z0)**2)

    radii.append(rr)

radii = np.concatenate(radii)


In [ ]:
radii_zhou[radii_zhou > 10] = np.abs(1000. - radii_zhou)[radii_zhou > 10]
radii[radii > 10] = np.abs(1000. - radii)[radii > 10]

bins = np.arange(0,2.55,0.05)

fig, ax = plt.subplots(1, 1, figsize=(8,6))

ax.hist(radii_zhou, bins=bins, alpha=0.5, density=True, label="Zhou+ 2021", color="blue")
ax.hist(radii, bins=bins, density=True, label="my model", color="black", histtype="step", hatch="////")
ax.legend()

ax.set_xlim(-0.05,2.5)
ax.set_xlabel("radial distance ($h^{-1}$ Mpc)")
ax.set_ylabel("relative number of LRG satellites")

plt.show()


In [ ]:
# #-- plot NFW profile of select host halos in given catalog

# def dens_prof(r, central):
#     M    = np.array(central["mvir"].data)
#     rvir = np.array(central["rvir"].data)
#     rs   = np.array(central["rs"].data)
    
#     phalo = M / ( (4/3)*np.pi*rvir**3 )
#     c     = rvir / rs
#     ANFW  = np.log(1 + c) - c / (1 + c)
    
#     x = np.array(r) / np.array(rvir)
    
#     return phalo / ( 3*ANFW*x*(1./c + x)**2 )

# rr = np.logspace(-1,4,100)

# fig, ax = plt.subplots(1, 1, figsize=(10,10))

# for i in range(50):
#     ax.loglog(rr, dens_prof(rr, galcat[galcat["upid"]==-1][i]), color="blue", alpha=0.05)

# ax.set_xlim(0.1,1e4)
# ax.set_ylim(1,1e10)

# plt.show()



# Halotools

In [ ]:
# %%time

# import halotools
# from halotools.sim_manager import RockstarHlistReader, CachedHaloCatalog #,TabularAsciiReader

# input_fname = "/Users/aberti/Desktop/research/sims/mdpl2/CATALOGS/hlist_0.61420.list"

# output_fname = "/Users/aberti/Desktop/research/sims/mdpl2/CATALOGS/a0p61420_vpeakmin125.hdf5"

# cols_to_keep = {"halo_id":(1,"<i8"), "halo_pid":(5,"<i8"), "halo_upid":(6,"<i8"), "halo_mvir":(10,"<f8"),
#                 "halo_rvir":(11,"<f8"), "halo_rs":(12,"<f8"), "halo_x":(17,"<f8"), "halo_y":(18,"<f8"),
#                 "halo_z":(19,"<f8"), "halo_vx":(20,"<f8"), "halo_vy":(21,"<f8"), "halo_vz":(22,"<f8"),
#                 "halo_vpeak":(62,"<f8"),}

# row_cut_min_dict = {"halo_vpeak":125}

# columns_to_convert_from_kpc_to_mpc = ("halo_rvir", "halo_rs")

# #-- metadata
# simname       = "mdpl2"
# halo_finder   = "rockstar"
# version_name  = "vpeakmin125"
# redshift      = 0.62813
# Lbox          = 1000.
# particle_mass = 1.51e9

# reader = RockstarHlistReader(input_fname, cols_to_keep, output_fname, simname, halo_finder, redshift,
#                              version_name, Lbox, particle_mass, row_cut_min_dict=row_cut_min_dict) 


In [ ]:
# %%time

# halocat = reader.read_halocat(columns_to_convert_from_kpc_to_mpc, write_to_disk=False, update_cache_log=False)

# #-- doesn't work
# # halocat = CachedHaloCatalog(simname=simname, halo_finder=halo_finder, version_name=version_name, redshift=redshift)



In [ ]:
%%time

#-- catalog columns
halo_id   = halocat["id"]
halo_pid  = halocat["pid"]
halo_upid = halocat["upid"]
halo_mvir = halocat["mvir"]
halo_rvir = halocat["rvir"]
halo_rs   = halocat["rs"]
halo_x    = halocat["x"]
halo_y    = halocat["y"]
halo_z    = halocat["z"]
halo_vx   = halocat["vx"]
halo_vy   = halocat["vy"]
halo_vz   = halocat["vz"]
halo_nfw_conc = halocat["rvir"]/halocat["rs"]


#-- metadata
simname       = "mdpl2"
halo_finder   = "rockstar"
version_name  = "vpeakmin125"
redshift      = 0.62813
Lbox          = 1000.
particle_mass = 1.51e9


In [ ]:
%%time

from halotools.sim_manager import UserSuppliedHaloCatalog

kwargs = dict(redshift=redshift, Lbox=Lbox, particle_mass=particle_mass, halo_id=halo_id, halo_pid=halo_pid,
              halo_mvir=halo_mvir, halo_x=halo_x, halo_y=halo_y, halo_z=halo_z, halo_upid=halo_upid, 
              halo_vx=halo_vx, halo_vy=halo_vy, halo_vz=halo_vz,
              halo_rvir=halo_rvir, halo_rs=halo_rs, halo_nfw_conc=halo_nfw_conc)

halo_catalog = UserSuppliedHaloCatalog(**kwargs)


In [ ]:
from halotools.empirical_models import HodModelFactory
from halotools.empirical_models import TrivialPhaseSpace, Zheng07Cens
from halotools.empirical_models import NFWPhaseSpace, Zheng07Sats

cens_occ_model  = Zheng07Cens(redshift=redshift)
cens_prof_model = TrivialPhaseSpace(redshift=redshift)

sats_occ_model  = Zheng07Sats(redshift=redshift)
sats_prof_model = NFWPhaseSpace(redshift=redshift)

model_instance = HodModelFactory(
        centrals_occupation = cens_occ_model,
        centrals_profile = cens_prof_model,
        satellites_occupation = sats_occ_model,
        satellites_profile = sats_prof_model)

# The model_instance is a composite model
# All composite models can directly populate N-body simulations
# with mock galaxy catalogs using the populate_mock method:

model_instance.populate_mock(halo_catalog, redshift=redshift)
